# 05_train_transformer

Fine-tune DistilBERT (or RoBERTa) using Hugging Face Trainer.

In [ ]:
# pip install transformers datasets accelerate evaluate
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np, os
model_name='distilbert-base-uncased'
out='artifacts/transformer'
os.makedirs(out, exist_ok=True)
ds = load_dataset('imdb')
tok = AutoTokenizer.from_pretrained(model_name)
def preprocess(x): return tok(x['text'], truncation=True, padding='max_length', max_length=256)
tds = ds.map(preprocess, batched=True)
tds = tds.remove_columns(['text']).set_format('torch')
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
training_args = TrainingArguments(output_dir=out, evaluation_strategy='epoch', per_device_train_batch_size=8, num_train_epochs=1)
def compute_metrics(p):
    import evaluate
    metric = evaluate.load('accuracy')
    preds = np.argmax(p.predictions, axis=1)
    return metric.compute(predictions=preds, references=p.label_ids)
trainer = Trainer(model=model, args=training_args, train_dataset=tds['train'].select(range(4000)), eval_dataset=tds['test'].select(range(1000)), tokenizer=tok, compute_metrics=compute_metrics)
trainer.train()
trainer.save_model(out)
